In [1]:
n0 = 2000
n1 = n0 + 2000

In [2]:
import pandas as pd
import random
import numpy as np

In [3]:
authorship = pd.read_csv('G1_mentee_authorship.csv')

In [4]:
target_author = pd.read_csv('target_author.csv')['author_id'].tolist()

In [5]:
len(target_author)

6949

In [6]:
authorship.head(5)

,work_id,author_position,author_id,institution_id,first_pub_year,last_pub_year,career_len,publication_year,scientific_age
0,W1033085884,first,A3022368640,I5388228,2006,2006,0,2006,0
1,W1033085884,middle,A2613136029,I5388228,2003,2010,7,2006,3
2,W1033085884,middle,A2044805072,I5388228,2006,2006,0,2006,0
3,W1033085884,middle,A2886369024,I5388228,2005,2006,1,2006,1
4,W1033085884,middle,A1016359704,I5388228,2006,2006,0,2006,0


In [7]:
len(set(authorship.work_id))

903183

In [8]:
inst_completeness = pd.read_csv('G1_mentee_inst_completeness.csv')

In [9]:
target_works1 = set(inst_completeness['work_id'][inst_completeness['ratio_inst.']>0])
len(target_works1)

751272

In [10]:
authorship = authorship[authorship.work_id.isin(target_works1)]

In [11]:
def group_info(author_i,df,backward_window,scientific_age_window):
    auship = df
    
    # Method2
    #mentor's all last-author publications
    works_i = auship['work_id'][(auship.author_id==author_i)&(auship.author_position=='last')]
    

    auship_i = auship[auship.work_id.isin(works_i)].sort_values(by='publication_year')
    # print('len(auship_i) : ',len(auship_i))

    auship_null = auship_i[~auship_i.institution_id.isnull()]

    # print('the ratio of with institution : ','{:.0%}'.format(len(auship_null)/len(auship_i)))
    
    years_i = auship_i.publication_year
    try:
        start_year = min(years_i)
        end_year = max(years_i)
    except:
        start_year = 2021
        end_year = 2022
    # print('start_year : ',start_year)
    # print('end_year : ',end_year)
    
    Group = pd.DataFrame([],columns=['work_id','author_position', 'author_id', 'institution_id', 'publication_year',
                                               'first_pub_year', 'scientific_age', 'group_size', 'mentor_id', 'current_year', 'ratio_institution'])
   
    for current_year in range(start_year,end_year+1):
        # print('current_year : ',current_year)
        auship_i_back = auship_i[(auship_i.publication_year>=current_year-backward_window)&
                                (auship_i.publication_year<=current_year)]
        
        auship_i_back =  auship_i_back.reset_index().drop(columns=['index'])
        #new consideration : current_scientific_age
        csa = np.array(auship_i_back['first_pub_year'] - current_year)*-1
        auship_i_back['current_scientific_age']  = csa

        auship_i_back_notnull = auship_i_back[~auship_i_back.institution_id.isnull()]
        
        
        if len(auship_i_back)==0:
            ratio_institution = 0
        else:
            ratio_institution = len(auship_i_back_notnull)/len(auship_i_back)
            
            # print('the ratio of with institution : ','{:.0%}'.format(ratio_institution))
        
        
        # ratio_institution = len(auship_i_back_notnull)/len(auship_i_back)
        # print('the ratio of with institution : ','{:.0%}'.format(ratio_institution))

        if not ratio_institution==0:
            try:
                institution_i = auship_i_back['institution_id'][auship_i_back.author_id==author_i].mode().iloc[0]
                # print('institution_i : ',institution_i)

                group_i = auship_i_back[(auship_i_back.institution_id==institution_i)
                                       &(auship_i_back.current_scientific_age<=scientific_age_window) # see above: current_scientific_age
                                       &(auship_i_back.author_id!=author_i)]

                group_size = len(set(group_i.author_id))
            # print('group_size : ',group_size)
            except:
                group_size = 0
            if group_size>0:
                group_i = group_i.assign(group_size = group_size)
                group_i = group_i.assign(mentor_id = author_i)
                group_i = group_i.assign(current_year = current_year)
                group_i = group_i.assign(ratio_institution = ratio_institution)
            else:
                group_i = pd.DataFrame([],columns=['work_id','author_position', 'author_id', 'institution_id', 'publication_year',
                                               'first_pub_year', 'scientific_age', 'group_size', 'mentor_id', 'current_year', 'ratio_institution'])
        else:
            group_i = pd.DataFrame([],columns=['work_id','author_position', 'author_id', 'institution_id', 'publication_year',
                                               'first_pub_year', 'scientific_age', 'group_size', 'mentor_id', 'current_year', 'ratio_institution'])
        Group = pd.concat((Group,group_i))
    return Group

In [12]:
backward_window = 5
scientific_age_window = 10
Data = pd.DataFrame([],columns=['work_id','author_position', 'author_id', 'institution_id', 'publication_year',
                                               'first_pub_year', 'scientific_age', 'group_size', 'mentor_id', 'current_year', 'ratio_institution'])

for i in range(n0,n1):
    author_i = target_author[i]
    data = group_info(author_i,authorship,backward_window,scientific_age_window)
    Data = pd.concat((Data,data))
    if i%10==0:
        print(i)

Data['current_scientific_age'] = Data['current_year'] - Data['first_pub_year']
Data.to_csv(f'groupsize/author_{n0}_{n1}.csv',index=0)

2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2220
2230
2240
2250
2260
2270
2280
2290
2300
2310
2320
2330
2340
2350
2360
2370
2380
2390
2400
2410
2420
2430
2440
2450
2460
2470
2480
2490
2500
2510
2520
2530
2540
2550
2560
2570
2580
2590
2600
2610
2620
2630
2640
2650
2660
2670
2680
2690
2700
2710
2720
2730
2740
2750
2760
2770
2780
2790
2800
2810
2820
2830
2840
2850
2860
2870
2880
2890
2900
2910
2920
2930
2940
2950
2960
2970
2980
2990
3000
3010
3020
3030
3040
3050
3060
3070
3080
3090
3100
3110
3120
3130
3140
3150
3160
3170
3180
3190
3200
3210
3220
3230
3240
3250
3260
3270
3280
3290
3300
3310
3320
3330
3340
3350
3360
3370
3380
3390
3400
3410
3420
3430
3440
3450
3460
3470
3480
3490
3500
3510
3520
3530
3540
3550
3560
3570
3580
3590
3600
3610
3620
3630
3640
3650
3660
3670
3680
3690
3700
3710
3720
3730
3740
3750
3760
3770
3780
3790
3800
3810
3820
3830
3840
3850
3860
3870
3880
3890
3900
3910
3920
3930
3940
3950
3960
3970
3980
3990
